In [ ]:
import os, sys
import argparse
from biobb_io.api.pdb import pdb
from biobb_amber.pdb4amber.pdb4amber_run import pdb4amber_run
from biobb_amber.leap.leap_gen_top import leap_gen_top
from biobb_amber.pmemd.pmemd_mdrun import pmemd_mdrun
from biobb_amber.process.process_minout import process_minout
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb
from biobb_amber.leap.leap_solvate import leap_solvate
from biobb_amber.leap.leap_add_ions import leap_add_ions
from biobb_amber.process.process_mdout import process_mdout
from biobb_amber.pmemd.pmemd_mdrun import pmemd_mdrun

import json

`create_dictionaries()` extracts the inputs parameters and arguments for the entire simulation from an inputs.json file. This file would contain the MD 
parameters for different stages, force field options and some input file names needed for solvation. 

In [ ]:
def create_dictionaries(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    dictionaries = {}
    for key in data.keys():
        dictionaries[key] = data[key]
    return dictionaries

In [ ]:
inputs_dict = create_dictionaries('../example/1aki/inputs.json')

`prepPDB()` prepares the input PDB file and processes it to make compatible with AMBER software. Disulfide bonds, missing hydrogen atoms etc. are added in this step. The force field chosen for the simulation is provided here an an input argument. 

In [ ]:
def prepPDB(input_PDB, forcefield, destination="./"):
        
    ## Clean the PDB file using pdb4amber tool
    output_pdb4amber_path = os.path.join(destination,'structure.pdb4amber.pdb')
    pdb4amber_run(input_pdb_path=input_PDB, output_pdb_path=output_pdb4amber_path)
    
    ## Create topology files for the protein. 
    leap_gen_top (input_pdb_path=output_pdb4amber_path,
                  output_pdb_path=os.path.join(destination,'structure.leap.pdb'), 
                  output_top_path=os.path.join(destination,'structure.leap.top'), 
                  output_crd_path=os.path.join(destination,'structure.leap.crd'), 
                  properties=forcefield)

`mdrun()` performs different MD simulation runs, including minimization, NVT, NPT equilibrations and production runs (free runs). The binary file used for the run can be changed to `pmemd` or `pmemd.cuda` depending upon whether you would want to run MD simulations on CPUs or GPUs. If you want to use GPUs, then choose `path_to_amber/22/bin/pmemd.cuda` as the binary path. 

Refer to this page for details: https://biobb-amber.readthedocs.io/en/latest/pmemd.html#module-pmemd.pmemd_mdrun  

In [ ]:
def mdrun (stage, input_top, prop, destination="./"):

    ## PMEMD_mdrun for any steps, including Minimization, Heating, NVT, NPT, and Production
    output_traj_path = os.path.join(destination,f"pmemd.{stage}.nc")
    output_rst_path = os.path.join(destination,f"pmemd.{stage}.rst")
    output_log_path = os.path.join(destination,f"pmemd.{stage}.log")
    
    print(output_traj_path, output_rst_path, output_log_path)

    ## Create and launch bb
    pmemd_mdrun(input_top_path=os.path.join(destination,input_top["top"]),
                 input_crd_path=os.path.join(destination,input_top["crd"]),
                 input_ref_path=os.path.join(destination,input_top["crd"]),
                 output_traj_path=output_traj_path,
                 output_rst_path=output_rst_path,
                 output_log_path=output_log_path,
                 properties=prop)

`solvation ()` is used to solvate the system using TIP3P. Howver, different water molecule formats could be used. 

Refer to this page for details: https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_solvate  

In [ ]:
def solvation (input_top, prop, destination='./'):
    
    ## Create and launch bb
    amber_to_pdb(input_top_path=os.path.join(destination,input_top["top"]),
                 input_crd_path=os.path.join(destination,input_top["crd"]),
                 output_pdb_path=os.path.join(destination,input_top["ambpdb"]),
                )

    ## Solvation of the protein
    leap_solvate(input_pdb_path=os.path.join(destination,input_top["ambpdb"]),
                 output_pdb_path=os.path.join(destination,'structure.solv.pdb'),
                 output_top_path=os.path.join(destination,'structure.solv.parmtop'),
                 output_crd_path=os.path.join(destination,'structure.solv.crd'),
                 properties=solvate_prop)

`addIons()` is used to add ions to neutralize the system and in addition create an buffer with prescribed concentration. These inputs are taken from the inputs.json file. 

Refer to this page for details: https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_add_ions  

In [ ]:
def addIons (input_PDB, prop, destination='./'):
    
    # Create prop dict and inputs/outputs
    output_ions_pdb_path = 'structure.ions.pdb'
    output_ions_top_path = 'structure.ions.parmtop'
    output_ions_crd_path = 'structure.ions.crd'

    # Create and launch bb
    leap_add_ions(input_pdb_path=input_PDB,
                  output_pdb_path=os.path.join(destination, output_ions_pdb_path),
                  output_top_path=os.path.join(destination,output_ions_top_path),
                  output_crd_path=os.path.join(destination,output_ions_crd_path),
                  properties=prop)

Force field can be different here. Instead of ff19SB, one could also use ff14SB. In addition, if new ligands are used, one can use gaff2 as well. 

In [ ]:
forcefield = {
    "forcefield" : ["protein.ff19SB"]
}
prepPDB ("../example/1aki/1aki.pdb", forcefield, destination="../example/1aki/")

In [ ]:
input_top = {
    "top": "structure.leap.top",
    "crd": "structure.leap.crd",
    "pdb": "structure.leap.pdb",
    "ambpdb": "structure.ambpdb.pdb"
}

prop = {
    'simulation_type' : "min_vacuo",
    "binary_path": "path_to_amber/22/bin/pmemd",
    "mdin" : { 
        'maxcyc' : 500,
        'ntpr' : 5,
        'ntr' : 1,
        'restraintmask' : '\":*&!@H=\"',
        'restraint_wt' : 50.0
    }
}

In [ ]:
mdrun ("min_vacuo", input_top, prop, destination="../example/1aki/")

In [ ]:
solvate_prop = {
    "forcefield" : ["protein.ff19SB"],
    "water_type": "TIP3PBOX",
    "distance_to_molecule": "9.0",   
    "box_type": "truncated_octahedron"
}

In [ ]:
solvation (input_top, solvate_prop, destination='../example/1aki/')

In [ ]:
ions_prop = {
    "forcefield" : ["protein.ff19SB"],
    "neutralise" : True,
    "positive_ions_type": "Na+",
    "negative_ions_type": "Cl-",
    "ionic_concentration" : 150, # 150mM
    "box_type": "truncated_octahedron"
}

addIons ('../example/1aki//structure.solv.pdb', ions_prop, destination='../example/1aki/')

In [ ]:
input_ions_top = {
    "top": "structure.ions.parmtop",
    "crd": "structure.ions.crd",
    "pdb": "structure.ions.pdb"
}

prop_min = {
    "simulation_type" : "minimization",
    "binary_path": "path_to_amber/22/bin/pmemd",
    "mdin" : { 
        'maxcyc' : 300, # Reducing the number of minimization steps for the sake of time
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 50.0                       # With a force constant of 50 Kcal/mol*A2
    }
}



In [ ]:
mdrun ("min", input_ions_top, prop_min, destination="../example/1aki/")

In [ ]:
input_heat_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.min.log",
    "rst": "pmemd.min.rst",
    "crd": "pmemd.min.rst"
}

prop_heat = {
    "simulation_type" : "heat",
    "binary_path": "path_to_amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 2500, # Reducing the number of steps for the sake of time (5ps)
        'ntr' : 1,       # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 10.0                       # With a force constant of 10 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("heat", input_heat_top, prop_heat, destination="../example/1aki/")

In [ ]:
input_nvt_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.heat.log",
    "rst": "pmemd.heat.rst",
    "crd": "pmemd.heat.rst"
}


prop_nvt = {
    "simulation_type" : 'nvt',
    "binary_path": "path_to_amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 50000, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 5.0,                            # With a force constant of 5 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("nvt", input_nvt_top, prop_nvt, destination="../example/1aki/")

In [ ]:
input_npt_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.nvt.log",
    "rst": "pmemd.nvt.rst",
    "crd": "pmemd.nvt.rst"
}

prop_npt = {
    "simulation_type" : 'npt',
    "binary_path": "path_to_amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 50000, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 2.5                               # With a force constant of 2.5 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("npt", input_npt_top, prop_npt, destination="../example/1aki/")

In [ ]:
input_free_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.npt.log",
    "rst": "pmemd.npt.rst",
    "crd": "pmemd.npt.rst"
}

prop_free = {
    "simulation_type" : 'free',
    "binary_path": "path_to_amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 250000, # Reducing the number of steps for the sake of time (5ps)
        'ntwx' : 500  # Print coords to trajectory every 500 steps (1 ps)
    }
}

In [ ]:
mdrun ("free", input_free_top, prop_free, destination="../example/1aki/")

### Gaussian Accelerated Molecular Dynamics (GaMD):

After the equilibrium MD simulation, we can perform enhanced sampling using GaMD. This feature is not currently implemented in the official BioBB_AMBER package. Be sure to include the necessary patches to run this simulation.

In [ ]:
input_gamd_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.free.log",
    "rst": "pmemd.free.rst",
    "crd": "pmemd.free.rst"
}

prop_gamd = {
    "simulation_type" : 'free',
    "binary_path": "path_to_amber/22/bin/pmemd.cuda",
    "mdin" : { 
        "nstlim" : 300000,
        "ntwx" : 500,
        "igamd" : 15,
        "iE" : 1,
        "iEP" : 1, 
        "iED" : 1,
        "irest_gamd" : 0,                                           
        "ntcmd" : 50000, 
        "nteb" : 250000, 
        "ntave" : 10000,
        "ntcmdprep" : 20000,
        "ntebprep" : 20000,
        "sigma0P" : 6.0,
        "sigma0D" : 6.0
    }
}

In [ ]:
def mdrun_gamd (stage, input_top, prop, destination="./"):

    ## PMEMD_mdrun for any steps, including Minimization, Heating, NVT, NPT, and Production
    output_traj_path = os.path.join(destination,f"pmemd.{stage}.nc")
    output_rst_path = os.path.join(destination,f"pmemd.{stage}.rst")
    output_log_path = os.path.join(destination,f"pmemd.{stage}.log")
    output_gamd_path = os.path.join(destination,f"pmemd.test.gamd.log")
    output_mdinfo_path = os.path.join(destination,f"pmemd.{stage}.mdinfo")
    
    print(output_traj_path, output_rst_path, output_log_path, output_mdinfo_path)

    ## Create and launch bb
    pmemd_mdrun(input_top_path=os.path.join(destination,input_top["top"]),
                 input_crd_path=os.path.join(destination,input_top["crd"]),
                 input_ref_path=os.path.join(destination,input_top["crd"]),
                 output_traj_path=output_traj_path,
                 output_rst_path=output_rst_path,
                 output_log_path=output_log_path,
                 output_gamd_path=output_gamd_path,
                 properties=prop)

In [ ]:
mdrun_gamd ("gamd", input_gamd_top, prop_gamd, destination="../example/1aki/")

In [ ]:
cd ../example/1aki

In [ ]:
!/path_to_amber/amber/22/bin/pmemd.cuda -O -i gamd.in -p structure.ions.parmtop -c pmemd.free.rst -o pmemd.gamd.out -x pmemd.gamd.nc -r pmemd.gamd.rst -gamd gamd-1.log

In [ ]:
from biobb_analysis.ambertools.cpptraj_slice import cpptraj_slice
prop = {
    'start': 1,
    'end': -1,
    'steps': 1000,
    'mask': 'all-atoms',
    'format': 'nc'
}
cpptraj_slice(input_top_path='structure.ions.parmtop',
            input_traj_path='pmemd.gamd.nc',
            output_cpptraj_path='pmemd.gamd.sliced.nc',
            properties=prop)